Сгенерируем датасет для оценки качества выполнения задачи. Возможно дообучим pytorch nn.

In [3]:
!pip install pymupdf

In [3]:
!pip install fpdf

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40713 sha256=3d030d429e9eb0115ed48106e615e2162d4ee5f59fce765845635860f679b694
  Stored in directory: c:\users\dsash\appdata\local\pip\cache\wheels\44\35\8b\86ce00cec7e4d13c5f189680ae0fa82f919bedc066c2cddae9
Successfully built fpdf
Note: you may need to restart the kernel to use updated packages.


In [24]:
import os
import cv2
import fitz
import random
import string
import numpy as np
import pandas as pd
from PIL import Image
from fpdf import FPDF

# Функция для генерации случайного текста или цифр
def generate_random_data(length):
    data_type = random.choice(["letters", "digits"])
    if data_type == "digits":
        return ''.join(random.choice(string.digits) for _ in range(length))
    else:
        return ''.join(random.choice(string.ascii_letters) for _ in range(length))

# Функция для генерации случайной таблицы с разным числом строк и столбцов
def generate_random_table():
    num_rows = random.randint(1, 10)  # Случайное число строк от 1 до 10
    num_cols = random.randint(1, 8)  # Случайное число столбцов от 1 до 10

    table_data = []
    for _ in range(num_rows):
        row = [generate_random_data(random.randint(1, 5)) for _ in range(num_cols)]
        table_data.append(row)

    return table_data

# Функция для генерации случайных искажений перспективы
def generate_perspective_transform():
    width, height = 210, 297  # Размеры листа (A4)
    max_perspective = 0.02  # Максимальное значение искажения перспективы
    tl = [random.uniform(0, max_perspective) * width, random.uniform(0, max_perspective) * height]
    tr = [random.uniform(1 - max_perspective, 1) * width, random.uniform(0, max_perspective) * height]
    bl = [random.uniform(0, max_perspective) * width, random.uniform(1 - max_perspective, 1) * height]
    br = [random.uniform(1 - max_perspective, 1) * width, random.uniform(1 - max_perspective, 1) * height]
    src = np.float32([tl, tr, bl, br])
    dst = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
    perspective_transform = cv2.getPerspectiveTransform(src, dst)
    return perspective_transform

# Функция для добавления шума к изображению
def add_noise(image, probability):
    if random.random() < probability:
        noise = np.random.normal(0, 25, image.shape).astype(np.uint8)
        noisy_image = cv2.add(image, noise)
        return noisy_image
    return image

# Функция для создания датасета
def generate_dataset(num_samples, noise_probability):
    # Создание директории "dataset", если она не существует
    directory = "dataset"
    if os.path.exists(directory):
        for file_name in os.listdir(directory):
            file_path = os.path.join(directory, file_name)
            os.remove(file_path)
    else:
        os.makedirs(directory)

    for i in range(num_samples):
        # Генерация случайной таблицы
        table_data = generate_random_table()
        df = pd.DataFrame(table_data)

        # Создание PDF файла с таблицей
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)

        for row in table_data:
            for item in row:
                pdf.cell(20, 10, str(item), border=1)
            pdf.ln()

        # Сохраним pdf файл
        pdf_file = f"dataset/sample_table_{i}.pdf"
        pdf.output(pdf_file)

        # Теперь считаем файл, чтобы наложить на него шум и искажение как на изображение
        doc = fitz.open(pdf_file)
        page = doc.load_page(0)
        pix = page.get_pixmap()
        img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
        img = np.ascontiguousarray(img[..., [2, 1, 0]])
        doc.close()

        # Создание искаженного изображения таблицы
        perspective_transform = generate_perspective_transform()
        img = cv2.warpPerspective(img, perspective_transform, (img.shape[1], img.shape[0]))
        img = add_noise(img, noise_probability)
        img = Image.fromarray(img)

        # Сохранение искаженного изображения в формате pdf
        image_file = f"dataset/sample_table_{i}.pdf"
        img.save(image_file)

        # Создание CSV файла-метки
        csv_file = f"dataset/sample_table_{i}.csv"
        df.to_csv(csv_file, index=False)

generate_dataset(10, 0.01)